# Twitter Sentiment Analysis using Natural Language Processing.

### Getting Necessary Data..

In [59]:
from nltk.corpus import twitter_samples
from nltk.tag import pos_tag


positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
text = twitter_samples.strings('tweets.20150430-223406.json')
tweet_tokens = twitter_samples.tokenized('positive_tweets.json')   # tokenizing positive tweets

### Lemmatizing

In [66]:
from nltk.stem.wordnet import WordNetLemmatizer

def lemmatize_sentence(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    for word, tag in pos_tag(tokens):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatized_sentence.append(lemmatizer.lemmatize(word, pos))
    return lemmatized_sentence

print(lemmatize_sentence(tweet_tokens[0]))

['#FollowFriday', '@France_Inte', '@PKuchly57', '@Milipol_Paris', 'for', 'be', 'top', 'engage', 'member', 'in', 'my', 'community', 'this', 'week', ':)']


### Removing Noise ( Cleaning stopwords , punctuations )

In [67]:
import re, string

def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens


In [68]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

print(remove_noise(tweet_tokens[0], stop_words))


['#followfriday', 'top', 'engage', 'member', 'community', 'week', ':)']


### Follow Through all the steps i.e Tokeninzing , Cleaning.

In [69]:

positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []

for tokens in positive_tweet_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

for tokens in negative_tweet_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

    
print(positive_tweet_tokens[300])
print(positive_cleaned_tokens_list[300])



['Stats', 'for', 'the', 'day', 'have', 'arrived', '.', '2', 'new', 'followers', 'and', 'NO', 'unfollowers', ':)', 'via', 'http://t.co/xxlXs6xYwe', '.']
['stats', 'day', 'arrive', '2', 'new', 'follower', 'unfollowers', ':)', 'via']


### Determining Word Density.

In [70]:
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token

all_pos_words = get_all_words(positive_cleaned_tokens_list)
all_pos_words

<generator object get_all_words at 0x000000001B7B87C8>

In [71]:
from nltk import FreqDist

freq_dist_pos = FreqDist(all_pos_words)
print(freq_dist_pos.most_common(10))


[(':)', 3691), (':-)', 701), (':d', 658), ('thanks', 388), ('follow', 357), ('love', 333), ('...', 290), ('good', 283), ('get', 263), ('thank', 253)]


### Preparing Data For Model.

In [72]:
def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)


### Splitting the Dataset for Training & Testing the Model

In [73]:
import random

positive_dataset = [(tweet_dict, "Positive")
                     for tweet_dict in positive_tokens_for_model]

negative_dataset = [(tweet_dict, "Negative")
                     for tweet_dict in negative_tokens_for_model]

dataset = positive_dataset + negative_dataset

random.shuffle(dataset)

train_data = dataset[:7000]
test_data = dataset[7000:]


### Building & Testing the Model

In [75]:
from nltk import classify
from nltk import NaiveBayesClassifier
classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy is:", classify.accuracy(classifier, test_data))

print(classifier.show_most_informative_features(10))


Accuracy is: 0.995
Most Informative Features
                      :( = True           Negati : Positi =   2047.4 : 1.0
                      :) = True           Positi : Negati =   1679.0 : 1.0
                     sad = True           Negati : Positi =     54.6 : 1.0
                     via = True           Positi : Negati =     39.7 : 1.0
                     bam = True           Positi : Negati =     21.4 : 1.0
                    glad = True           Positi : Negati =     18.0 : 1.0
                     x15 = True           Negati : Positi =     16.0 : 1.0
                followed = True           Negati : Positi =     15.1 : 1.0
                  arrive = True           Positi : Negati =     14.1 : 1.0
                   didnt = True           Negati : Positi =     12.1 : 1.0
None


# ...END...